<a href="https://colab.research.google.com/github/MSBX5420/team-mount-massive/blob/master/PySpark_Simple_Top_5_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install pyspark

     |████████████████████████████████| 217.8MB 62kB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=63332ed1ab799d3c63c5b851677111e72ad00141f4adb0e5c0398690571e5a60
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
import pyspark
from pyspark.sql import SparkSession

sc = SparkContext()
sqlContext = SQLContext(sc)

spark=SparkSession.builder.appName('Recommend_Function').getOrCreate()


ValueError: ignored

In [0]:
import pandas as pd

In [25]:
movies = spark.read.csv('drive/My Drive/Data/sample_movies_single.csv', header =True, inferSchema=True)

+---------+--------+--------------------+------+--------+--------+----------+---------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|  titleId|ordering|               title|region|language|   types|attributes|isOriginalTitle|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+--------+--------------------+------+--------+--------+----------+---------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0000012|       7|L'arrivée d'un tr...|    \N|      \N|original|        \N|              1|tt0000012|    short|The Arrival of a ...|L'arrivée d'un tr...|    0.0|     1896|     \N|             1|Action,Documentar...|          7.4|  9435.0|
|tt0000582|       4|             Ben

Load in data and create a copy.

Create a new Weighted Rating Column in data copy using formula cited to be from IMDB on Kaggle (though IMDB says it keeps its weighted rating formula private to maintain efficacy). This formula takes number of votes for the movie, the average rating over all movies, the average rating for the movie, and an inputed value representing the number of votes requred to make the list (I chose movies over the 10th percentile since we already created a cutoff in the data cleaning process).  

In [0]:
movies.show(10)

In [0]:
from pyspark.sql.functions import *

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, col

In [0]:
mean = movies.groupBy().avg("averageRating").take(1)[0][0]
movies = movies.withColumn("AverageTotalVotes", lit(mean))

In [48]:
movies = movies.withColumn("WeightedRating", lit(0).cast(IntegerType()))
movies = movies.withColumn("MinimumVotes", lit(100).cast(IntegerType()))
movies.show(10)

+---------+--------+--------------------+------+--------+--------+----------+---------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+--------------+------------+-----------------+
|  titleId|ordering|               title|region|language|   types|attributes|isOriginalTitle|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|WeightedRating|MinimumVotes|AverageTotalVotes|
+---------+--------+--------------------+------+--------+--------+----------+---------------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+--------------+------------+-----------------+
|tt0000012|       7|L'arrivée d'un tr...|    \N|      \N|original|        \N|              1|tt0000012|    short|The Arrival of a ...|L'arr

In [0]:
def weighted(v, m, C, R) : 
  return (v/(v+m) * R) + (m/(m+v) * C)


In [0]:
weighted_udf = udf(weighted, IntegerType())

In [0]:
movies = movies.withColumn('WeightedRating', weighted_udf('numVotes', 'MinimumVotes', 'AverageTotalVotes', 'averageRating'))

In [58]:
movies.show(10)

IllegalArgumentException: ignored

Create a function that takes three parameters: genre, type, and age of film. This function cycles through films meeting the criteria, and produces a top 5 list based on highest weighted rating. 

In [38]:
def recommend(genrechoice : str) :
  PossMovie = movies.filter(movies.genres.like(genrechoice)
  CASE 
    WHEN (agechoice = 'New') THEN PossMovie = PossMovie.filter(movies.startYear >= 2000)
    WHEN (agechoice = 'Old') THEN PossMovie = PossMovie.filter(movies.startYear < 2000) 
    ELSE PossMovie = PossMovie
  END
  PossMovie = PossMovie.select('title', 'genres', 'startYear', 'titleType', 'WeightedRating')
  PossMovie.show(10)

SyntaxError: ignored

In [0]:
  Ordered = PossMovie.sort_values(by="WeightedRating", ascending = False) 
  Ordered = Ordered[['title', 'genres', 'titleType', 'startYear', 'WeightedRating']]
  Top5 = Ordered[:5]

Test function:

In [0]:
recommend('Comedy', 'movie', 'New')

,title,genres,titleType,startYear,WeightedRating
769,Marmoulak,"Comedy,Drama",movie,2004,8.499704
663,Dil Chahta Hai,"Comedy,Drama,Romance",movie,2001,8.099953
1135,Guardians of the Galaxy,"Action,Adventure,Comedy",movie,2014,7.999997
1248,Zootopia,"Adventure,Animation,Comedy",movie,2016,7.999993
1337,En man som heter Ove,"Comedy,Drama,Romance",movie,2015,7.699945
